In [5]:
import os
from datetime import datetime
import requests
import tiktoken
from typing import Dict, Tuple
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
headers = {
    'Content-Type': 'application/json'
}
freshdesk_api_key = os.environ.get('FRESHDESK_API_KEY')
auth = (freshdesk_api_key, 'X')
domain = 'nueca'

import requests

tickets_url = f"https://{domain}.freshdesk.com/api/v2/tickets?order_by=created_at&order_type=desc&per_page=3"
ticket_response = requests.get(tickets_url, headers=headers, auth=auth)
tickets = ticket_response.json() if ticket_response.status_code == 200 else []

In [7]:
def parse_freshdesk_conversations(conversations, ticket_description) -> dict:
    sorted_conversations = sorted(conversations, key=lambda x: x['created_at'])
    
    full_text_conversation = ""
    full_conversation = []
    customer_conversation = ""
    agent_conversation = ""
    ordered_conversation_details = []

    message_data = {
        'id': ticket_description['id'],
        'sequence': 1,
        'body_text': ticket_description['description_text'],
        'timestamp': ticket_description['created_at'],
        'user_id': ticket_description['requester_id'],
        'role': 'Customer'
    }

    message = f'Customer: {message_data['body_text']}'
    full_text_conversation += message + "\n"
    full_conversation.append(message)
    customer_conversation += message_data['body_text'] + "\n"
    ordered_conversation_details.append(message_data)

    for i, conv in enumerate(sorted_conversations, 2):
        message_data = {
            'id': conv['id'],
            'sequence': i,
            'body_text': conv['body_text'],
            'timestamp': conv['created_at'],
            'user_id': conv['user_id'],
        }

        if message_data['body_text'] == '':
            continue
        if conv['incoming']:
            message = f'Customer: {message_data['body_text']}'
            full_text_conversation += message + "\n" 
            full_conversation.append(message)
            message_data['role'] = 'Customer'
            customer_conversation += message_data['body_text'] + "\n"
        else:
            message = f'Agent: {message_data['body_text']}'
            full_text_conversation += message + "\n" 
            full_conversation.append(message)
            message_data['role'] = 'Agent'
            agent_conversation += message_data['body_text'] + "\n"

        ordered_conversation_details.append(message_data)
        
    return {
        'full_text_conversation': full_text_conversation,
        'full_conversation': full_conversation,
        'customer_conversation': customer_conversation,
        'agent_conversation': agent_conversation,
        'ordered_conversation_details': ordered_conversation_details,
    }

In [8]:
def get_all_conversations(ticket_id, ticket_description):
    conversations = []
    url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}/conversations"

    while url:
        response = requests.get(url, headers=headers, auth=auth)
        if response.status_code != 200:
            break
        data = response.json()
        conversations.extend(data)
        link = response.headers.get('link')
        if link and 'rel="next"' in link:
            import re
            match = re.search(r'<([^>]+)>;\s*rel="next"', link)
            url = match.group(1) if match else None
            print("has pagination")
        else:
            url = None

    parsed_conversations = parse_freshdesk_conversations(conversations, ticket_description)

    return parsed_conversations

In [9]:
def add_to_map(mmap: Dict[int, Tuple[int, int]], key: int, value: Tuple[int, int]):
    if key in mmap:
        existing = mmap[key]
        mmap[key] = (existing[0] + value[0], existing[1] + value[1])
    else:
        mmap[key] = value

    return mmap

In [18]:
mmap: Dict[int, Tuple[int, int]] = {}
mmap

{}

In [ ]:
# model = "gpt-4o-mini"
# encoding = tiktoken.encoding_for_model(model)

# customer_tokens = encoding.encode(customer_messages)
# agent_tokens = encoding.encode(agent_messages)

# 11810

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


for ticket_id in range(7713, 11810):
    tickets_url = f"https://nueca.freshdesk.com/api/v2/tickets/{ticket_id}"
    ticket_description = requests.get(tickets_url, headers=headers, auth=auth).json()

    date_str = ticket_description['created_at']
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    month = dt.month

    parsed_conversations = get_all_conversations(ticket_id, ticket_description)
    
    customer_messages = parsed_conversations['customer_conversation']
    customer_length = len(customer_messages)
    
    agent_messages = parsed_conversations['agent_conversation']
    agent_length = len(agent_messages)

    mmap = add_to_map(mmap, month, (customer_length, agent_length))
    
    print(f'ID: {ticket_id} - {mmap}')
    
    print("-----")

ID: 7713 - {12: (270, 132)}
-----
ID: 7714 - {12: (270, 132), 1: (128, 464)}
-----
ID: 7715 - {12: (270, 132), 1: (129, 464)}
-----
ID: 7716 - {12: (270, 132), 1: (252, 464)}
-----
ID: 7717 - {12: (270, 132), 1: (436, 464)}
-----
ID: 7718 - {12: (270, 132), 1: (487, 1450)}
-----
ID: 7719 - {12: (270, 132), 1: (600, 1619)}
-----
ID: 7720 - {12: (270, 132), 1: (754, 2054)}
-----
ID: 7721 - {12: (270, 132), 1: (1152, 2413)}
-----
ID: 7722 - {12: (270, 132), 1: (1252, 2413)}
-----
ID: 7723 - {12: (270, 132), 1: (1324, 2709)}
-----
ID: 7724 - {12: (270, 132), 1: (1363, 2709)}
-----
ID: 7725 - {12: (270, 132), 1: (1646, 3070)}
-----
ID: 7726 - {12: (270, 132), 1: (1803, 3212)}
-----
ID: 7727 - {12: (270, 132), 1: (1855, 3322)}
-----
ID: 7728 - {12: (270, 132), 1: (2084, 3920)}
-----
ID: 7729 - {12: (270, 132), 1: (2632, 4879)}
-----
ID: 7730 - {12: (270, 132), 1: (2840, 5046)}
-----
ID: 7731 - {12: (270, 132), 1: (3085, 5963)}
-----
ID: 7732 - {12: (270, 132), 1: (3159, 6518)}
-----
ID: 7733

In [ ]:
tokens:

get problem
rag classification
rag


search results 5 fetch
average length 

'created_at': '2025-07-03T00:55:24Z'


tickets_url = f"https://nueca.freshdesk.com/api/v2/tickets/7133"
ticket_description = requests.get(tickets_url, headers=headers, auth=auth).json()

date_str = ticket_description['created_at']
dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
month = dt.month
month

date_str